<a href="https://colab.research.google.com/github/VMutafchiev/LLMs/blob/main/Hermes_3_0_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq langchain==0.0.228 --progress-bar off
!pip install -qqq chromadb==0.3.26 --progress-bar off
!pip install -qqq sentence-transformers==2.2.2 --progress-bar off
!pip install -qqq auto-gptq==0.2.2 --progress-bar off
!pip install -qqq einops==0.6.1 --progress-bar off
!pip install -qqq unstructured==0.8.0 --progress-bar off
!pip install -qqq transformers==4.30.2 --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off

In [ ]:
from pathlib import Path

import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, GenerationConfig, TextStreamer, pipeline

from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
import argparse

from sentence_transformers import SentenceTransformer

import warnings

In [ ]:
questions_dir = Path("tlc")
questions_dir.mkdir(exist_ok=True, parents=True)

def write_file(question, answer, file_path):
    text = f"""
Q: {question}
A: {answer}
""".strip()
    with Path(questions_dir / file_path).open("w") as text_file:
        text_file.write(text)

In [ ]:
write_file(
    question="It is safe to use thinkorswim ® Sharing?",
    answer="""In designing the thinkorswim sharing system, we were very aware of the security concerns some users may have so we took several steps to ensure your data is protected. The technology for sharing has been built into thinkorswim since early 2013; we have performed an internal security review and all of the data required to create the Sharing links is handled by our technology via our secure backend servers, not on your local computer or the public Internet. Only data that is validly shared from inside the thinkorswim application can be communicated via the sharing links themselves.

When a user clicks on a Shared link, the only information that is passed from the browser to the thinkorswim application is the sharing link pointer; all other data is delivered by our secure servers directly to the thinkorswim application. This ensures only valid data is retrieved and shared. Your account and other personal information is not shared, but if you have a public MyTrade profile then that public display name is used.

myTrade is a service of myTrade, Inc., a separate but affiliated firm. TD Ameritrade is not responsible for the services of myTrade, or content shared through the service.""".strip(),
    file_path="qiestion_1.txt",
)
write_file(
    question="How do I delete a thinkorswim ® Sharing link?",
    answer="""All thinkorswim Sharing links cannot be redirected or deleted. If you'd like to remove a link from your thinkorswim platform, go to the Tools tab> Shared Items > and you can click on the 'X' next to the link you wish to remove. ‘Removing’ means you will no longer find the thinkorswim Sharing link in the "Shared Items" view.""".strip(),
    file_path="question_2.txt",
)

write_file(
    question="How do I add/remove a gadget to the sidebar?",
    answer="""To add another gadget, click on the "Plus" button at the lower left of the left-hand sidebar. This will open the “Select Gadget” menu. Click on any gadget name to add it to the left-hand sidebar. To change an existing gadget to a different type, click on the “Show Actions Menu" icon at the upper right of its top-right corner.

More info on the left sidebar""".strip(),
    file_path="qiestion_3.txt",
)

write_file(
    question="How do I switch between accounts?",
    answer="""At the upper left of the main thinkorswim window, there is a pane that says “Account”. Using the drop-down menu there, you can switch between any account linked to your current user name or a total account view which links every account for that user.""".strip(),
    file_path="qiestion_4.txt",
)

write_file(
    question="How do I hide account information?",
    answer="""If you are in a semi-public location or showing the platform to a friend, you do have the option to hide your personal financial data while still using the platform. To activate this feature, click on the down arrow to the right of your “Option Buying Power” or “Net Liq & Day Trades” in the “ACCOUNT INFO” panel. In the screen that pops-up, there is a switch towards the upper right that says “PRIVACY”. Click the switch, and all of your specific account data will be hidden throughout the platform. Click the switch again to restore the values.

More info on Account Info""".strip(),
    file_path="qiestion_5.txt",
)
write_file(
    question="How do I import/create/delete/edit watchlists?",
    answer="""The watchlists that you have created in any other TD Ameritrade platform should be visible within the thinkorswim trading platform. For example, if you have created a watchlist on the website, it should appear under the “Personal” section of your watchlists. To create a new watchlist click on the MarketWatch Tab, and click on “Quotes." Click on "Show Actions Menu" icon at the upper right-hand corner of the sub-tab and select the option to Create Watchlist. In the next window, you will be prompted to input a name for your watchlist. You can either add the ticker symbols you wish to monitor manually by typing the symbol in the blank box or paste symbols from the clipboard. You can also upload a .CSV or Excel file, then click import. Once you are done, click Save.

More info on watchlists""".strip(),
    file_path="qiestion_6.txt",
)
write_file(
    question="How to sort, unsort, and customize watchlists?",
    answer="""You are able to click and drag any security to where you wish within the watchlist. To sort the list in ascending order (A-Z) for example, left click on the column header “Symbol” directly below the title of the watchlist. To unsort the list, left click the tab “Symbol” again. If the list is sorted in ascending order a shaded gray arrow in the tab “Symbol” will be pointing up. The arrow will be pointing down if the list is sorted in descending order (Z-A). This rule applies to any of the column headers on the list.


To customize the columns, right click on any of the column headers to reveal a dropdown. Select “Customize” and a window will appear with a selection of available columns on the left side. To add a column, select the column title you prefer and then click the “Add Item” button below the list. To remove an item from your current set of columns, select the column title you prefer and then click the “Remove Item" button below the list. You can load the default columns by clicking the “Load Defaults” button on the upper right of the window. The “Restore” button will reload the column set you started with at the time you opened the customization window. When you are done making changes, click "OK".

More info on watchlists:""".strip(),
    file_path="qiestion_7.txt",
)
write_file(
    question="Can I get rid of the left sidebar or get it back?",
    answer="""Certainly. If you look at the vertical bar dividing the left sidebar from the rest of the platform, in the center, there is a small arrow pointing left. Click on this arrow and your sidebar will be minimized until this arrow is clicked again.

More info on the left sidebar: HERE.
Where do I find the messages that popped up (reject messages, chat room alerts, etc.)?

To find previous messages, click on the envelope icon labeled “Messages” at the top left corner of the screen.

Additionally, you can view your messages in the Message Center gadget on the left sidebar. More info on the Message Center:""".strip(),
    file_path="qiestion_8.txt",
)
write_file(
    question="What resources do you offer to help me learn the platform?",
    answer="""We offer a variety of resources, you can find educational content by clicking on the “Education” tab (Access by clicking Education>Learning Center), in addition to the “Help” tab of the platform (Access by clicking Help>Contact Us). Depending on how you like to receive content, one or a combination of the following should offer the information you need to maximize the power of this award-winning platform:


1.    User Manual- the User Manual is a 50+ page document that will walk you through all the ins and outs of the platform. Learn how to trade, monitor, analyze, find trading opportunities, and customize your trading experience. To access the manual, click HERE.
2.    Video Tutorials, Articles, and Glossary. All found right here on this website!
3.    FAQ’s- You’re reading them!
4.    Seminars, both live and archived. Live seminars can be found HERE. You can find archived items by clicking Support/Chat at the upper right-hand corner of the platform and select “Seminars”.
5.    Videos on https://tdameritradenetwork.com/. Platform demo: HERE.
6.    thinkScript® Manual can be found HERE.""".strip(),
    file_path="qiestion_9.txt",
)
write_file(
    question="How do I fund my account?",
    answer="""You can fund your account through an Electronic ACH, wire, check, ACAT transfer, internal transfer or by depositing securities. To get started, log into your account at www.tdameritrade.com and navigate to Accounts>Deposit/Withdraw. Depending on the method you choose to fund the account, appropriate instructions will be provided to you accordingly. You will find more details here: https://www.tdameritrade.com/funding-and-transfers.page
How do I set a default account on the platform?

If you have multiple accounts under a particular username and wish to change the default account, please go to the upper right-corner of the platform and select Setup>Application Settings>General>Account and click the drop down box next to "Default account" to make your selection. Once you are done, click "Apply Settings". You will then want to be sure and save your workspace so these settings are retained. Go to Setup>Save Workspace As… to save a new workspace or resave an existing workspace.""".strip(),
    file_path="qiestion_10.txt",
)
write_file(
    question="Where can I check the margin requirements/trading hours for Futures?",
    answer="""You can view trading hours and other futures product specs on the TD Ameritrade website.

To view margin requirements, login to the thinkorswim platform and go to a symbol entry box (either in the Trade tab or the Charts tab). You will see a down arrow to the right of the entry field; click it to reveal the symbol table. Here click on the Futures button to see things like the symbol, description, tick size, tick value, and initial margin.""".strip(),
    file_path="qiestion_11.txt",
)
write_file(
    question="How do I link my accounts?",
    answer="""You would do this through the TD Ameritrade website. Log into the account then follow Client Services>My Profile>Link Accounts>Link my accounts>Grant access. Please note, you must accept the Account Linking Agreement prior to submitting an access request. Once entered, the person getting the link request would have to log in and follow the same steps to approve any linking requests pending review.""".strip(),
    file_path="qiestion_12.txt",
)
write_file(
    question="How do I change my account nickname?",
    answer="""To change nicknames, please log in at www.tdameritrade.com > navigate to Client Services > My Profile > Link Accounts > Manage linked accounts, then select “Edit nickname” next to the account that you would like to change, enter the new name, and save.""".strip(),
    file_path="qiestion_13.txt",
)
write_file(
    question="Where can I download the platform?",
    answer="""Please download the platform by logging into your account at www.tdameritrade.com and click on Trade>Trading Platforms>thinkorswim>Download thinkorswim.""".strip(),
    file_path="qiestion_14.txt",
)
write_file(
    question="How do I add or remove the Home Screen?",
    answer="""Click on Setup>Application Settings>General>Startup>Screen shown on startup, then select one of the listed options and click ‘Apply settings’.""".strip(),
    file_path="qiestion_15.txt",
)
write_file(
    question="How do I change my nickname in the chatrooms? ",
    answer="""Please click on Setup>Application Settings>My profile>Nickname, once you are done making changes click ‘Apply settings’.""".strip(),
    file_path="qiestion_16.txt",
)
write_file(
    question="What does PDT stand for?",
    answer="""PDT stands for ‘pattern day trader’.  If you make more than 3 day trades in any 5 business day period, you will be flagged as a pattern day trader.  As long as you maintain a net liquidating value above $25,000, there will not be any adverse effects to your account.’.""".strip(),
    file_path="qiestion_17.txt",
)
write_file(
    question="What is Net Liq? ",
    answer="""Net Liq, or Net Liquidating Value, represents the value of your account if all stock positions were liquidated at the mark.

With regards to options, the mark price used is the mid-point between the bid and ask.

The Mark Price for other underlying instruments is calculated by the following formula:

    If BID and ASK are both NaN or zero, or inverted, then it is equal Last Price
    If Last is between BID and ASK, then it is equal to Last Price
    If LAST is above ASK, then it is equal to ASK
    If LAST is below BID, then it is equal to BID""".strip(),
    file_path="qiestion_18.txt",
)
write_file(
    question="What is Level II?",
    answer="""Level II displays the best prices for each of the exchanges making markets in stocks and options.""".strip(),
    file_path="qiestion_19.txt",
)
write_file(
    question="What is Time and Sales?",
    answer="""Time and Sales displays a record for every trade that happened for the specified product intraday including a record of time, price and volume of each transaction.""".strip(),
    file_path="qiestion_20.txt",
)
write_file(
    question="What is Portfolio Margin?",
    answer="""A specific type of account margining that bases the account’s minimum requirements and available buying power upon the overall position risk in the account. Portfolio margining involves a great deal more risk than cash accounts and is not suitable for all investors. Contact TD Ameritrade at 800-669-3900 for more details""".strip(),
    file_path="qiestion_21.txt",
)
write_file(
    question="What is MMM? ",
    answer="""MMM, or Market Maker Move, measures the expected magnitude of price movement based on market volatility.  It measures the implied move based on volatility differential between the front and back month.  The MMM helps to identify the implied move due to an event between now and the front month expiration (if an event exists).""".strip(),
    file_path="qiestion_22.txt",
)
write_file(
    question="What is implied volatility?",
    answer="""Implied volatility is an estimate of the volatility of the underlying stock that is derived from the market value of an option.  Implied volatility is the volatility number that, if plugged into a theoretical pricing model along with all the other inputs, would yield a theoretical value of an option equal to the market price of the same option.""".strip(),
    file_path="qiestion_23.txt",
)
write_file(
    question="What is Futures Intraday Margining?",
    answer="""Futures Intraday Margining is the reduced holding requirement as compared to the initial or overnight margin requirement for futures trading available intraday.  The rate is generally 25% of the margin requirement for a particular product but can change at any time without notice.  The minimum net liquidation value of $15,000 is required to be eligible.  Intraday margin applies between the hours of 8:30 a.m. CT and 3 p.m. CT.""".strip(),
    file_path="qiestion_24.txt",
)
write_file(
    question="What is the Product Depth Curve?",
    answer="""The Product Depth Curve is a graphical representation of the market price of each contract month for any given futures product. We display this as price over the expiration month of the product.  This allows you to easily display Inverted and Normal curves in the futures product that you choose.""".strip(),
    file_path="qiestion_25.txt",
)

In [ ]:
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda:0'

In [ ]:
model_name_or_path = "TheBloke/Nous-Hermes-13B-GPTQ"
model_basename = "nous-hermes-13b-GPTQ-4bit-128g.no-act.order"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        device=DEVICE,
        use_triton=use_triton,
        quantize_config=None)

In [ ]:
generation_config = GenerationConfig.from_pretrained(model_name_or_path)
generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "pad_token_id": 0,
  "transformers_version": "4.30.2"
}

In [ ]:
question = (
    "Which is better Java or Python?"
)
prompt = f"""
### Instructions: {question}
### Response:
""".strip()

In [ ]:
%%time
input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to(DEVICE)
with torch.inference_mode():
  output = model.generate(
      inputs=input_ids,
      temperature=0.5,
      max_new_tokens=512)


CPU times: user 8.07 s, sys: 562 ms, total: 8.64 s
Wall time: 11.2 s


In [ ]:
streamer = TextStreamer(
    tokenizer,
    skip_prompt=True,
    skip_special_tokens=True,
    use_multiprocessing=False
)

In [ ]:
# Inference can also be done using transformers' pipeline
# Prevent printing spurious transformers error when using pipeline with AutoGPTQ
logging.set_verbosity(logging.CRITICAL)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=2048,
    temperature=0,
    top_p=0.9,
    repetition_penalty=1.2,
    generation_config=generation_config,
    streamer=streamer,
    batch_size=1,
)

In [ ]:
llm = HuggingFacePipeline(pipeline=pipe)
responce = llm(prompt)

Both Java and Python are popular programming languages with their own strengths and weaknesses. It ultimately depends on the specific use case, personal preference, and expertise of the programmer.


In [ ]:
model = SentenceTransformer("embaas/sentence-transformers-multilingual-e5-base")

In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name='embaas/sentence-transformers-multilingual-e5-base',
    model_kwargs={"device":DEVICE}
)

In [ ]:
DEFAULT_TEMPLATE = """
### Instructions: You are thinkorswim chat agent with a name: Lucy an A.I TLC chat agent, that is talking to a customer.
Use only the chat history, and the following information{context}.
to answer in a helpful and polite manner to the question. If you don't know the answer - make an excuse and say that you don't know.
Keep your replies short, compassionate and informative.
{chat_history}
### Input: {question}
### Response:
""".strip()

class Chatbot:
    def __init__(
        self,
        text_pipeline: HuggingFacePipeline,
        embeddings: HuggingFaceEmbeddings,
        documents_dir: Path,
        prompt_template: str = DEFAULT_TEMPLATE,
        verbose: bool = False,
    ):
        prompt = PromptTemplate(
            input_variables=["context", "question", "chat_history"],
            template = prompt_template,
        )
        self.chain = self._create_chain(text_pipeline, prompt, verbose)
        self.db = self._embed_data(documents_dir, embeddings)

    def _create_chain(
        self,
        text_pipeline: HuggingFacePipeline,
        prompt: PromptTemplate,
        verbose: bool = False,
    ):
        memory = ConversationBufferMemory(
            memory_key = "chat_history",
            human_prefix = "### Input",
            ai_prefix = "### Response",
            input_key = "question",
            output_key = "output_text",
            return_messages=False,
        )
        return load_qa_chain(
            text_pipeline,
            chain_type="stuff",
            prompt=prompt,
            memory=memory,
            verbose=verbose,
        )

    def _embed_data(
        self, documents_dir: Path, embeddings: HuggingFaceEmbeddings
      ) -> Chroma:
          loader = DirectoryLoader(documents_dir, glob="**/*txt")
          documents = loader.load()
          text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=0)
          texts = text_splitter.split_documents(documents)
          return Chroma.from_documents(texts, embeddings)

    def __call__(self, user_input: str) -> str:
        docs = self.db.similarity_search(user_input)
        return self.chain.run({"input_documents": docs, "question": user_input})


In [ ]:
chatbot = Chatbot(llm, embeddings, "./tlc/")

In [ ]:
warnings.filterwarnings("ignore", category=UserWarning)

while True:
    user_input = input("Question: ")
    if user_input.lower() in ["bye", "goodbye", "see ya", "thank you bye"]:
        break
    answer = chatbot(user_input)
    print()

Question: hello
Hi there! How may I assist you today?

Question: what's your name?
My name is Lucy, how can I help you?

Question: what are you
I am a virtual assistant designed to provide support for our customers.

Question: bye
